In [1]:
#########H2_op.numqubit=2

from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

In [2]:
#######Parameterized circuit

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter

params = [Parameter("theta"), Parameter("phi"), Parameter("lam"), 
          Parameter("theta1"), Parameter("phi1"), Parameter("lam1")]

qr = QuantumRegister(2, name="q")
circ = QuantumCircuit(qr)
circ.u(params[0], params[1], params[2], qr[0])
circ.u(params[3], params[4], params[5], qr[1])
 
print(circ)

       ┌──────────────────┐ 
q_0: ──┤ U(theta,phi,lam) ├─
     ┌─┴──────────────────┴┐
q_1: ┤ U(theta1,phi1,lam1) ├
     └─────────────────────┘


In [3]:
from qiskit.primitives import Estimator
import numpy as np
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP
from qiskit.utils import algorithm_globals



In [265]:
estimator = Estimator()
optimizers = [SLSQP(maxiter=0)]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
    algorithm_globals.random_seed = 0
    
    ansatz = circ
    counts = []
    values = []
    
    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)
    
    k = np.random.rand(6)*np.pi*2 #pick random points
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point = k, callback=store_intermediate_result)
    
    result = vqe.compute_minimum_eigenvalue(operator=H2_op)
    
    print(result)
    converge_counts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)

print("\rOptimization complete      ");



# # ######################data saving
# existing_data = pd.read_csv('output2.csv')

# # New data
# new_data1 = result.optimal_point[0]
# new_data2 = result.optimal_point[1]
# new_data3 = result.optimal_point[2]
# new_data4 = result.optimal_point[3]
# new_data5 = result.optimal_point[4]
# new_data6 = result.optimal_point[5]
# new_data7 = result.eigenvalue
# # Create a new DataFrame for the new data
# df_new = pd.DataFrame({'Param1': [new_data1], 'Param2': [new_data2], 'Param3': [new_data3], 'Param4': [new_data4], 
#                        'Param5': [new_data5], 'Param6': [new_data6], 'Eigenvalue':[new_data7]})
# # Concatenate the existing and new dataframes
# df_combined = pd.concat([existing_data, df_new], ignore_index=True)

# # Save the combined DataFrame to the CSV file without column labels
# df_combined.to_csv('output2.csv', index=False)

Optimizer: SLSQP        {   'aux_operators_evaluated': None,
    'cost_function_evals': 7,
    'eigenvalue': -1.151028713547112,
    'optimal_circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fc2bd350d90>,
    'optimal_parameters': {   Parameter(lam): 4.942180456993617,
                              Parameter(phi): 4.282196304983521,
                              Parameter(theta1): 4.042913762630213,
                              Parameter(lam1): 3.3732780618537337,
                              Parameter(phi1): 1.0525740390738823,
                              Parameter(theta): 2.6614136001737934},
    'optimal_point': array([4.94218046, 3.37327806, 4.2821963 , 1.05257404, 2.6614136 ,
       4.04291376]),
    'optimal_value': -1.151028713547112,
    'optimizer_evals': None,
    'optimizer_result': <qiskit.algorithms.optimizers.optimizer.OptimizerResult object at 0x7fc2bdadac90>,
    'optimizer_time': 0.014505386352539062}
Optimization complete      


In [7]:
import pandas as pd
data1 = result.optimal_point[0]
data2 = result.optimal_point[1]
data3 = result.optimal_point[2]
data4 = result.optimal_point[3]
data5 = result.optimal_point[4]
data6 = result.optimal_point[5]
data7 = result.eigenvalue


df = pd.DataFrame({'Param1': [data1], 'Param2': [data2], 'Param3': [data3], 'Param4': [data4],
                   'Param5': [data5], 'Param6': [data6], 'Eigenvalue':[data7]})
df.to_csv('output2.csv', index=False)